### Part of Speech Tagging with Hidden Markov Models 

Using a Hidden Markov Model implementation from the [Pomegranate](http://pomegranate.readthedocs.io/) library to tag part of speech out of sentences from the Brown corpus

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown
import random

### 1: Read and preprocess the dataset

The dataset is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus), available in NLTK

In [3]:
nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]

[nltk_data] Downloading package brown to /home/robyeva/nltk_data...
[nltk_data]   Package brown is already up-to-date!


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

In [4]:
def train_test_splitting_function(data, train_test_split = 0.8):
    """Split the corpus in set of sentences belonging to train and test set. The train_test_split fraction determines 
    the size of the training set."""
    list_idx = list(range(len(data)))
    random.shuffle(list_idx)
    split = int(train_test_split * len(list_idx))

    training_data = np.array(data)[list_idx[:split]]
    testing_data = np.array(data)[list_idx[split:]]
    
    return training_data, testing_data

In [5]:
len(training_corpus.tagged_sents())

57340

In [7]:
data = training_corpus.tagged_sents()
data = list(data)
training_data, testing_data = train_test_splitting_function(data, train_test_split = 0.8)
print(training_data[0])

[('Hunting', 'VBG-HL'), ('rifles', 'NNS-HL'), (',', ',-HL'), ("'61", 'CD-HL')]


/home/robyeva/miniconda3/envs/nlp_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/robyeva/miniconda3/envs/nlp_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


### 2 - Set up counts 

Counts are used to define the emission and transition probabilities. In short, the HMM tagger has one hidden state for each possible tag, and is parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.


In [8]:
def pair_counts(data):
    """Return nested dictionary with the number of occurrences of a given (word, tag) combination.
    E.g. pair_counts[NOUN][time] == # occurrences in data of word "time", tagged as NOUN.
    """
    dic_data = {}
    data_list = [item for sublist in data for item in sublist]
    
    for i in range(len(data_list)):   
        try: 
            dic_data[data_list[i][1]][data_list[i][0]] += 1
        except:
            try: 
                dic_data[data_list[i][1]][data_list[i][0]] = 1
            except:
                dic_data[data_list[i][1]] = {}
                dic_data[data_list[i][1]][data_list[i][0]] = 1
    return dic_data 

In [9]:
def unigram_counts(data):
    """Return a dictionary with the number of occurrences of the value in data (across the whole data).
    E.g. unigram_dict[NOUN] == # words in the data tagged as NOUN.
    """
    tags_list = [item[1] for sublist in data for item in sublist]
    used = set()
    unique = [x for x in tags_list if x not in used and (used.add(x) or True)]
    unigram_dict = {}
    for el in unique:
        unigram_dict[el] = len([i for i, x in enumerate(tags_list) if x == el])
    return unigram_dict


In [10]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique pair of values with the number of occurrences of 
    successive tags.
    E.g. bigram_dict[(NOUN, VERB)] == # occurrences of tag NOUN followed by tag VERB in sequences.
    """
    biagram_dict = {}
    for idx in range(len(sequences)):

        c = sequences[idx]
        # gives the tuples of consecutive tags
        consec = [(c[i][1],c[i+1][1]) for i in range(0,len(c)-1)]
        
        for el in consec: 
            try: 
                biagram_dict[el] +=1
            except: 
                biagram_dict[el] = 1
    return biagram_dict


In [11]:
def starting_counts(sequences):
    """Return a dictionary that counts the number of occurrences of each tag at the beginning of any sentence in the sequence.
    E.g. tag_dict[NOUN] == # starting words tagged as NOUN
    """
    tag_dict = {}
    for idx in range(len(sequences)):
        # take start element only
        try:
            el = sequences[idx][0][1]
        except:
            continue
        try: 
            tag_dict[el] +=1
        except: 
            tag_dict[el] = 1
    return tag_dict

In [12]:
def ending_counts(sequences):
    """Return a dictionary that counts the number of occurrences of each tag at the end of any sentence in the sequence.
    E.g. tag_dict[NOUN] == #words at the end of sentence tagged as NOUN

    """
    tag_dict = {}
    for idx in range(len(sequences)):
        # take start element only
        try:
            el = sequences[idx][-1][1]
        except:
            continue
        try: 
            tag_dict[el] +=1
        except: 
            tag_dict[el] = 1
    return tag_dict

In [13]:
# calculate counts for training data
emission_counts = pair_counts(training_data)
tag_unigrams = unigram_counts(training_data)
tag_bigrams = bigram_counts(np.array(training_data))
tag_starts = starting_counts(np.array(training_data))
tag_ends = ending_counts(np.array(training_data))
train_counts = pair_counts(training_data)

### 3 - Set up the Hidden Markov Model

There are as many nodes as unique tags, plus a starting and an ending node. The **emission distribution** assigns a probability to each word, given a tag. The **transition probabilities** govern the transitions from one tag to another one (edges).

In [14]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")
states=dict()

words = [item[0] for sublist in training_data for item in sublist]
used = set()
vocabulary = [x for x in words if x not in used and (used.add(x) or True)]

tags_list = [item[1] for sublist in training_data for item in sublist]
used = set()
tags_unique = [x for x in tags_list if x not in used and (used.add(x) or True)]

for t in tags_unique:
    
    # add emission probabilities P(w|t)
    prob = {}
    for i, w in enumerate(vocabulary):
        try:
            prob[w] = train_counts[t][w] / float(tag_unigrams[t])
        except: 
            prob[w] = 0 
    prob_emissions = DiscreteDistribution(prob)
    
    states[t] = State(prob_emissions, name=t)
    basic_model.add_states(states[t])
    
    # add transition to the end state P(end|t)
    try:
        end_prob = tag_ends[t] / sum(tag_ends.values())
    except:
        end_prob = 0
    basic_model.add_transition(states[t], basic_model.end, end_prob)
    
    # add transition to the start state P(t|start) 
    try:
        start_prob = tag_starts[t] / sum(tag_starts.values())
    except:
        start_prob = 0
    basic_model.add_transition(basic_model.start, states[t], start_prob)      

# add transitions across states (from t1 to t2)
for t1 in tag_unigrams.keys():
    for t2 in tag_unigrams.keys():
        try:
            prob = tag_bigrams[t1,t2] / tag_unigrams[t1]
        except:
            prob = 0 
        basic_model.add_transition(states[t1],states[t2], prob)

In [15]:
# create the model
basic_model.bake()

In [22]:
print("In the model there are ", basic_model.state_countcount(), " states and ", basic_model.edge_count(), " edges")

In the model there are  348  states and  120408  edges


In [17]:
# visualize transition matrix
column_order = ["base-hmm-tagger-start"] + sorted(tags_unique) + ['base-hmm-tagger-end']  # Override the Pomegranate default order
column_names = [s.name for s in basic_model.states]
order_index = [column_names.index(c) for c in column_order]

# re-order the rows/columns to match the alphabetic column order 
transitions = basic_model.dense_transition_matrix()[:, order_index][order_index, :]
print("The state transition matrix, P(Xt|Xt-1):\n")
print(transitions)

The state transition matrix, P(Xt|Xt-1):

[[0.         0.00065399 0.00152599 ... 0.         0.03243809 0.        ]
 [0.         0.         0.11316063 ... 0.         0.0062867  0.00041444]
 [0.         0.         0.         ... 0.         0.00061644 0.00136266]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00213936 0.         0.00092062]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### 4 - Model performance

In [18]:
def replace_unknown(sequence, vocabulary):
    """Replace unknown words (i.e. not present in the vocabulary) by 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in vocabulary else 'nan' for w in sequence]

def simplify_decoding(X, vocabulary, model):
    """X should be a 1-D sequence of observations for the model to predict. We use the Viterbi algortihm to choose the 
    path with highest probability
    """
    _, state_path = model.viterbi(replace_unknown(X, vocabulary))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

In [19]:
def accuracy(X, Y, vocabulary, model):
    """Calculate the prediction accuracy.
    X: list of words
    Y: list of the true labels (tags)
    Vocabulary: list of all the unique words in the corpus
    Model: the baked HMM
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        try:
            most_likely_tags = simplify_decoding(observations, vocabulary, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [24]:
# separate training and test set in lists of words and tags
## Train set
corpus_words_training = []
for sentence in training_data:
    word_list = tuple([item[0] for item in sentence])
    corpus_words_training.append(word_list)

corpus_tags_training = []
for sentence in training_data:
    t_list = tuple([item[1] for item in sentence])
    corpus_tags_training.append(t_list)
    
## Test set
corpus_words_test = []
for sentence in testing_data:
    word_list = tuple([item[0] for item in sentence])
    corpus_words_test.append(word_list)

corpus_tags_test = []
for sentence in testing_data:
    t_list = tuple([item[1] for item in sentence])
    corpus_tags_test.append(t_list)

In [25]:
hmm_training_acc = accuracy(corpus_words_training, corpus_tags_training, vocabulary, basic_model)
print("training accuracy HMM model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(corpus_words_test, corpus_tags_test, vocabulary, basic_model)
print("testing accuracy HMM model: {:.2f}%".format(100 * hmm_testing_acc))

training accuracy HMM model: 97.48%
testing accuracy HMM model: 89.43%


### 5 - Decoding sequences with the HMM Tagger

In [27]:
corpus_words_test[3]

('It',
 'is',
 'puzzling',
 'to',
 'the',
 'occidental',
 'mind',
 '(',
 'to',
 'mine',
 'at',
 'least',
 ')',
 'to',
 'assign',
 '``',
 'sacredness',
 "''",
 'to',
 'animal',
 ',',
 'insect',
 ',',
 'and',
 'plant',
 'life',
 '.')

In [29]:
for idx in range(2):
    print("Sentence: {}\n".format(corpus_words_test[idx]))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(corpus_words_test[idx], vocabulary, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(corpus_tags_test[idx])
    print("\n")

Sentence: ('John', 'received', 'a', 'promotion', 'in', 'his', 'firm', '.')

Predicted labels:
-----------------
['NP', 'VBD', 'AT', 'NN', 'IN', 'PP$', 'NN', '.']

Actual labels:
--------------
('NP', 'VBD', 'AT', 'NN', 'IN', 'PP$', 'NN', '.')


Sentence: ('He', 'conducted', 'it', 'with', 'less', 'diplomacy', 'and', 'more', 'spontaneous', 'violence', 'than', 'the', 'Sicilians', ',', 'but', 'he', 'had', 'his', 'huge', 'North', 'Side', 'portion', 'to', 'exploit', 'and', 'he', 'made', 'a', 'great', 'deal', 'of', 'money', '.')

Predicted labels:
-----------------
['PPS', 'VBD', 'PPO', 'IN', 'AP', 'NN', 'CC', 'QL', 'JJ', 'NN', 'IN', 'AT', 'NN', ',', 'CC', 'PPS', 'HVD', 'PP$', 'JJ', 'JJ-TL', 'NN-TL', 'NN', 'TO', 'VB', 'CC', 'PPS', 'VBD', 'AT', 'JJ', 'NN', 'IN', 'NN', '.']

Actual labels:
--------------
('PPS', 'VBD', 'PPO', 'IN', 'AP', 'NN', 'CC', 'AP', 'JJ', 'NN', 'CS', 'AT', 'NPS', ',', 'CC', 'PPS', 'HVD', 'PP$', 'JJ', 'JJ-TL', 'NN-TL', 'NN', 'TO', 'VB', 'CC', 'PPS', 'VBD', 'AT', 'JJ', 'NN'

Disclaimer: Notebook is based on exercises of the Natural Language Processing Udacity Nanodegree.